In [2]:
from Organizer import Organizer


Researcher = Organizer()

data = await Researcher.search("can you generate a report on the research paper 'Attention is all you need'?")

data

can you generate a report on the research paper 'Attention is all you need'?


[{'title': 'A Report on “Attention Is All You Need” by Ashish Vaswani ...', 'description': 'Irecently delved into theresearch papertitled “AttentionIsAll You Need,” authored by Ashish Vaswani and his colleagues.', 'url': 'https://medium.com/@shivayapandey359/attention-is-all-you-need-26586e6ab8ca'}, {'title': 'Attention is All you Need', 'description': 'by A Vaswani·Cited by 136871—Anattentionfunctioncanbe described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output areallvectors.', 'url': 'https://papers.neurips.cc/paper/7181-attention-is-all-you-need.pdf'}, {'title': 'Attention Is All You Need', 'description': '"AttentionIsAll You Need" is a 2017 landmarkresearch paperin machine learning authored by eight scientists working at Google. Thepaperintroduced a new\xa0...', 'url': 'https://en.wikipedia.org/wiki/Attention_Is_All_You_Need'}, {'title': 'The background needed to understand "Attention is All You ...', 'description': 'In my op

'**Report: Attention is All You Need**\n=====================================\n\n**Introduction**\n---------------\n\nThe research paper "Attention is All You Need" introduces a new deep learning architecture called the Transformer, which is based on the attention mechanism proposed in 2014 by Bahdanau et al. The Transformer approach has become the main architecture of large language models like those based on GPT.\n\n**Authors**\n------------\n\nThe authors of the paper are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan Gomez, Lukasz Kaiser, and Illia Polosukhin. All eight authors were equal contributors to the paper.\n\n**Historical Context**\n---------------------\n\nThe paper builds on the work of previous researchers, including the Elman network (1990) and LSTM (1995). The attention mechanism was first proposed in 2014 by Bahdanau et al.\n\n**Key Contributions**\n----------------------\n\nThe paper makes several key contributions:\n\n*   The Transfo

In [3]:
print(data)

**Report: Attention is All You Need**

**Introduction**
---------------

The research paper "Attention is All You Need" introduces a new deep learning architecture called the Transformer, which is based on the attention mechanism proposed in 2014 by Bahdanau et al. The Transformer approach has become the main architecture of large language models like those based on GPT.

**Authors**
------------

The authors of the paper are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan Gomez, Lukasz Kaiser, and Illia Polosukhin. All eight authors were equal contributors to the paper.

**Historical Context**
---------------------

The paper builds on the work of previous researchers, including the Elman network (1990) and LSTM (1995). The attention mechanism was first proposed in 2014 by Bahdanau et al.

**Key Contributions**
----------------------

The paper makes several key contributions:

*   The Transformer architecture is designed to process all tokens in paralle

In [3]:
import json

json.loads('{"status":"success", "answer": "Cybercab", "urls":null}')


{'status': 'success', 'answer': 'Cybercab', 'urls': None}

**Report: Attention is All You Need**
=====================================

**Introduction**
---------------

The research paper "Attention is All You Need" introduces a new deep learning architecture called the Transformer, which is based on the attention mechanism proposed in 2014 by Bahdanau et al. The Transformer approach has become the main architecture of large language models like those based on GPT.

**Authors**
------------

The authors of the paper are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan Gomez, Lukasz Kaiser, and Illia Polosukhin. All eight authors were equal contributors to the paper.

**Historical Context**
---------------------

The paper builds on the work of previous researchers, including the Elman network (1990) and LSTM (1995). The attention mechanism was first proposed in 2014 by Bahdanau et al.

**Key Contributions**
----------------------

The paper makes several key contributions:

*   The Transformer architecture is designed to process all tokens in parallel, rather than sequentially.
*   The paper introduces a new attention mechanism that allows the model to focus on different parts of the input sequence.
*   The authors suggest using dropout and linearly scaling up the learning rate to stabilize training.

**Methodology**
----------------

The paper uses an encoder-decoder architecture, where the encoder is a Transformer and the decoder is a Transformer with a different set of weights. The paper uses self-attention to allow the model to attend to all positions in the input sequence simultaneously.

**Results**
------------

The paper shows that the Transformer approach can achieve state-of-the-art results on several machine translation tasks. The paper also shows that the Transformer approach can be parallelized, which allows it to be accelerated on GPUs.

**Impact**
------------

The paper has been cited over 100,000 times as of 2024. The Transformer architecture has become the main architecture of large language models like those based on GPT. The paper has had a significant impact on the field of natural language processing and machine learning.

**Conclusion**
----------

The paper concludes that the Transformer approach is a powerful and efficient way to process sequential data. The paper suggests that the Transformer approach can be used for a wide range of natural language processing tasks, including machine translation, question answering, and text summarization.

**Key Findings**
-----------------

*   The Transformer architecture outperforms traditional RNN-based models on machine translation tasks.
*   The attention mechanism is a key component of the Transformer architecture, which allows the model to focus on specific parts of the input sequence when generating the output sequence.
*   The Transformer architecture has the potential to be applied to a wide range of tasks, including language modeling, question answering, and text summarization.

**Recommendations**
-------------------

*   The Transformer architecture is a powerful tool for sequence modeling and generation, and it has the potential to be applied to a wide range of tasks.
*   The attention mechanism is a key component of the Transformer architecture, and it allows the model to focus on specific parts of the input sequence when generating the output sequence.
*   The Transformer architecture can be parallelized, which allows it to be accelerated on GPUs.


In [21]:
import re
def split_corpus(corpus, max_words=4000):
    
    
    corpus = corpus.replace('\n',' ')
    while "  " in corpus:
        corpus = corpus.replace("  ", ' ')
    words = corpus.split()
    all_splits = []
    
    # Iterate over the words and split them into chunks of max_words
    for i in range(0, len(words), max_words):
        all_splits.append(" ".join(words[i:i + max_words]))
    
    return all_splits

x = split_corpus(data)

In [24]:
len(x[0].split())

3529

In [25]:
x[0]

'Here are the extracted and summarized information related to "All colors of sunflowers" in bullet format: * **Colors of Sunflowers:** + Cream + Gold + Yellow + Orange + Red + Mahogany + Chocolate brown + Pink (note: there are no blue sunflowers in nature) * **Varieties of Sunflowers:** + \'Shock-O-Lat\' (deep, dark brown with golden tips) + \'Coconut Ice\' (creamy vanilla-white with black centers) + \'Prado Red Shades\' (chocolate-tinted centers with red petals) * **Characteristics:** + Easy to grow from seeds + Great for bees and other pollinators + Can be grown in tall or tiny varieties + Can be used in vases and bouquets + Can be used as a fun project for kids * **Tips:** + Plant seeds 6 inches deep for giant sunflowers + Pinch stems when 5 sets of leaves have formed for more flowers + Choose from a variety of colors and sizes to suit your needs Here are the extracted and summarized information related to "All colors of sunflowers" in bullet format within 200 words: **Colors of Sun